In [ ]:
import numpy as np
import time
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
Bernoulli = tf.contrib.distributions.Bernoulli

data = input_data.read_data_sets("MNIST_data/", one_hot=True).train
data.images[data.images>0.5] = 1.0
data.images[data.images<=0.5] = 0.0
#data.images[:,0:392] = (data.images[:,0:392]-.5)*2

In [ ]:
for i in range(14):
    for j in range(14):
        data.images[:,i*14+j] = data.images[:,56*i+2*j] 

In [ ]:
def plot_banded_error(errs, label, ax=None):
    ax = ax or plt
    if np.shape(errs)[0]==2:
        s = np.mean(np.std(errs,1), axis=0)
        line, = ax.plot(np.arange(1, len(s)+1), np.square(s), label=label, linewidth=3)
    else:
        m = np.mean(errs, axis=0)
    #    m = np.mean(np.reshape(m,[-1,10]),1)
        s = np.std(errs, axis=0)
        line, = ax.plot(np.arange(1, len(m)+1), m, label=label, linewidth=3)
    #    ax.fill_between(np.arange(1, len(m)+1), m-s, m+s, color=line.get_color(), alpha=0.075)

def fatlegend(ax, *a, **kw):
    leg = ax.legend(*a, **kw)
    for l in leg.legendHandles:
        l.set_linewidth(l.get_linewidth()*2.0)
    return leg

def jacobian(y_flat, x):
    n = y_flat.shape[0]
    loop_vars = [
        tf.constant(0, tf.int32),
        tf.TensorArray(tf.float32, size=n),
    ]
    _, jacobian = tf.while_loop(
        lambda j, _: j < n,
        lambda j, result: (j+1, result.write(j, tf.gradients(y_flat[j], x))),
        loop_vars)
    return tf.squeeze(jacobian.stack())

### Graph

In [ ]:
def get_weights(wn):
    with tf.variable_scope(tf.get_variable_scope(), reuse=True):
        return tf.get_variable(wn)

def add_layer(inputs, wn, in_dim, out_dim, af=None):
    try:
        w = tf.get_variable(wn, [out_dim, in_dim+1], initializer=tf.truncated_normal_initializer)
    except:
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            w = tf.get_variable(wn)
    inputs = tf.concat([inputs, tf.ones([tf.shape(inputs)[0],1])], 1)
    ww = tf.transpose(w)
    return tf.matmul(inputs, ww) if af is None else af(tf.matmul(inputs, ww))


likelihood_ratio = lambda f, p, s, w: opt.compute_gradients(tf.reduce_mean(
                                          tf.stop_gradient(f) * log_prob(tf.tile(p,[ns,1]),s), 0), w)
prob = lambda p, s: (tf.pow(tf.clip_by_value(p,eps,1), s)
                     * tf.pow(1-tf.clip_by_value(p,0,1-eps), 1-s))
log_prob = lambda p, s: tf.log(prob(p,s))

ts  = tf.placeholder(tf.float32)
lr  = tf.placeholder(tf.float32)
tau = tf.placeholder(tf.float32)

eps = 1e-7
dim = 392
#dim = 98
nn = 200
ns = tf.placeholder(tf.int32)

y_ = tf.placeholder(tf.float32, [None, dim])
x  = tf.placeholder(tf.float32, [None, dim])

h   = add_layer(x, "wxh", dim, nn, tf.nn.sigmoid)
sh  = tf.stop_gradient(tf.reshape(Bernoulli(probs=h, dtype=tf.float32).sample(ns),[-1,nn]))
h2  = add_layer(sh, "whh", nn, nn, tf.nn.sigmoid)
sh2 = tf.stop_gradient(tf.reshape(Bernoulli(probs=h2, dtype=tf.float32).sample(ns),[-1,nn]))
y   = add_layer(sh2, "why", nn, dim, tf.nn.sigmoid)

In [ ]:
yh  = -log_prob(y, tf.tile(y_,[ns**2,1]))
nll = tf.reduce_mean(tf.reduce_sum(yh,1))
yx  = tf.reduce_mean(tf.reshape(yh,[ns,-1,dim]),0)

opt = tf.train.AdamOptimizer(lr)
dy = opt.compute_gradients(nll, get_weights("why")) 
dh = likelihood_ratio(tf.reduce_sum(yh,1,keep_dims=True), h2, sh2, get_weights("whh"))
dx = likelihood_ratio(tf.reduce_sum(yx,1,keep_dims=True), h, sh, get_weights("wxh"))

gd = [list(i) for i in dy]+[list(i) for i in dx]
for i in range(2):
    gd[i][0] = tf.check_numerics(gd[i][0], str(i))
#gdn = (tf.norm(gd[2][0])+tf.norm(gd[1][0]))/2.
train = opt.apply_gradients(gd)

In [ ]:
def muProp(p, s, layer, wx, wh, wy):
    p_ = tf.concat([p, tf.ones([tf.shape(p)[0],1])], 1)
    if layer==1:
        p_ = tf.nn.sigmoid(tf.matmul(p_,tf.transpose(wh)))
        p_ = tf.concat([p_, tf.ones([tf.shape(p_)[0],1])], 1)
    z = tf.nn.sigmoid(tf.matmul(p_,tf.transpose(wy)))
    z = -log_prob(z, tf.tile(y_,[ns**(layer-1),1]))
    z = tf.reduce_sum(z,1,keep_dims=True)
    
    dfh = tf.gradients(z,p)[0]
    z, dfh, p = [tf.tile(i, [ns,1]) for i in [z, dfh, p]]
    g = z + tf.reduce_sum(dfh*(s-p),1,keep_dims=True)
    w = wx if layer==1 else wh
    mg = tf.stop_gradient(dfh)*p    
    dmg = tf.gradients(tf.reduce_mean(mg, 0), w)[0]
    return g, dmg

cvh = muProp(h2, sh2, 2, get_weights("wxh"), get_weights("whh"), get_weights("why"))
cvx = muProp(h, sh, 1, get_weights("wxh"), get_weights("whh"), get_weights("why"))

mdh = likelihood_ratio(tf.reduce_sum(yh,1,keep_dims=True)-cvh[0], h2, sh2, get_weights("whh"))
mdx = likelihood_ratio(tf.reduce_sum(yx,1,keep_dims=True)-cvx[0], h, sh, get_weights("wxh"))

mpgd = [list(i) for i in dy]+[list(i) for i in mdh]+[list(i) for i in mdx]
mpgd[1][0] += cvh[1]
mpgd[2][0] += cvx[1]
for i in range(3):
    mpgd[i][0] = tf.check_numerics(mpgd[i][0], str(i))
    
mpgdn = [tf.norm(mpgd[2][0]), tf.norm(mpgd[1][0])]
mptrain = opt.apply_gradients(mpgd)

In [ ]:
def mpidb(z, w, n, f, out):
    s1 = add_layer(z, w+"_", n, nn/2, tf.nn.tanh)
    s2 = add_layer(s1, w, nn/2, out, tf.nn.tanh)
    loss = tf.square(tf.stop_gradient(f)-s2)
    return loss, s2

def sProp(p, layer, wx, wh, wy):
    if layer==1:
        s = mpidb(p, "widbh", nn, sh2-h2, nn)[1]
        p = tf.concat([p, tf.ones([tf.shape(p)[0],1])], 1)        
        p = tf.nn.sigmoid(tf.matmul(p, tf.transpose(wh))) + s
    p = tf.concat([p, tf.ones([tf.shape(p)[0],1])], 1)    
    z = tf.nn.sigmoid(tf.matmul(p,tf.transpose(wy)))
    z = -log_prob(z, tf.tile(y_,[ns**(layer-1),1]))
    return tf.tile(z,[ns,1])

bh = mpidb(sh, "widbh", nn, sh2-h2, nn)
bx = mpidb(x, "widbx", dim, sh-h, nn)

cvh = sProp(bh[1]+h2, 2, get_weights("wxh"), get_weights("whh"), get_weights("why"))
cvx = sProp(bx[1]+h, 1, get_weights("wxh"), get_weights("whh"), get_weights("why"))

spdh = likelihood_ratio(tf.reduce_sum(yh-cvh,1,keep_dims=True), h2, sh2, get_weights("whh"))
spdx = likelihood_ratio(tf.reduce_sum(yx-cvx,1,keep_dims=True), h, sh, get_weights("wxh"))

spgd = [list(i) for i in dy]+[list(i) for i in spdh]+[list(i) for i in spdx]
for i in range(3):
    spgd[i][0] = tf.check_numerics(spgd[i][0], str(i))
spgdn = (tf.norm(spgd[2][0])+tf.norm(spgd[1][0]))/2.
sptrain = opt.apply_gradients(spgd)

optt = tf.train.AdamOptimizer(lr*10, beta2=.9)
ibtrain = optt.minimize(bx[0]+bh[0])

In [ ]:
baselineh = tf.placeholder(tf.float32)
baselinex = tf.placeholder(tf.float32)

def idb(z, w, n, f, out):
    s1 = add_layer(z, w+"_", n, nn/2, tf.nn.tanh)
    s2 = add_layer(s1, w, nn/2, out)
    loss = tf.square(tf.stop_gradient(f)-s2)
    return loss, s2

bsh = tf.cond(ts<1, lambda: tf.reduce_mean(yh),
                    lambda: .999*baselineh + .001*tf.reduce_mean(yh))
bsx = tf.cond(ts<1, lambda: tf.reduce_mean(yx),
                    lambda: .999*baselinex + .001*tf.reduce_mean(yx))

bx = idb(x, "widbx", dim, yx-bsx, 1)
bh = idb(sh, "widbh", nn, yh-bsh, 1)

bdh = likelihood_ratio(tf.reduce_sum(yh-bsh-bh[1],1,keep_dims=True), h2, sh2, get_weights("whh"))
bdx = likelihood_ratio(tf.reduce_sum(yx-bsx-bx[1],1,keep_dims=True), h, sh, get_weights("wxh"))

bgd = [list(i) for i in dy]+[list(i) for i in bdh]+[list(i) for i in bdx]
for i in range(3):
    bgd[i][0] = tf.check_numerics(bgd[i][0], str(i))
bgdn = (tf.norm(bgd[2][0])+tf.norm(bgd[1][0]))/2.
btrain = opt.apply_gradients(bgd)

optt = tf.train.AdamOptimizer(lr*10, beta2=.99999)
ibtrain = optt.minimize(bx[0]+bh[0])

In [ ]:
def sample_baseline(f, layer):
    n = 0 if layer==1 else 1
    sf = tf.reduce_sum(tf.reshape(f,[ns,-1,dim]),n, keep_dims=True)
    sf = tf.reshape(tf.tile(sf,[1,ns,1]), [-1,dim])
    return (sf-f)/(tf.cast(ns,tf.float32)-1)

ayh = sample_baseline(yh,2)
ayx = sample_baseline(yx,1)

bdh = likelihood_ratio(tf.reduce_sum(yh-ayh,1,keep_dims=True), h2, sh2, get_weights("whh"))
bdx = likelihood_ratio(tf.reduce_sum(yx-ayx,1,keep_dims=True), h, sh, get_weights("wxh"))

bgd = [list(i) for i in dy]+[list(i) for i in bdh]+[list(i) for i in bdx]
for i in range(3):
    bgd[i][0] = tf.check_numerics(bgd[i][0], str(i))
bgdn = (tf.norm(bgd[2][0])+tf.norm(bgd[1][0]))/2.
btrain = opt.apply_gradients(bgd)

In [ ]:
def straight_through(p):
    s = tf.stop_gradient(tf.reshape(Bernoulli(probs=p,dtype=tf.float32).sample(ns),[-1,nn])-p)+p
    p2 = add_layer(s, "whh", nn, nn, tf.nn.sigmoid)
    s2 = tf.stop_gradient(tf.reshape(Bernoulli(probs=p2,dtype=tf.float32).sample(ns),[-1,nn])-p2)+p2
    sy = -log_prob(add_layer(s2, "why", nn, dim, tf.nn.sigmoid), tf.tile(y_,[ns**2,1]))
    return tf.reduce_mean(tf.reduce_sum(sy,1))

sttrain = opt.minimize(straight_through(h))


def Gumbel_dist(p):
    g0 = -tf.log(-tf.log(tf.random_uniform(tf.shape(p))))
    g1 = -tf.log(-tf.log(tf.random_uniform(tf.shape(p))))
    u1 = tf.exp((tf.log(tf.clip_by_value(p,eps,1))+g1)/tau)
    u0 = tf.exp((tf.log(1-tf.clip_by_value(p,0,1-eps))+g0)/tau)
    return u1/(u0+u1+eps)

def Gumbel(p):
    s = Gumbel_dist(p)
    p2 = add_layer(s, "whh", nn, nn, tf.nn.sigmoid)
    s2 = Gumbel_dist(p2)
    sy = -log_prob(add_layer(s2, "why", nn, dim, tf.nn.sigmoid), tf.tile(y_,[ns**2,1]))
    return tf.reduce_mean(tf.reduce_sum(sy,1))

gbtrain = opt.minimize(Gumbel(h))

In [ ]:
wxh = tf.placeholder(tf.float32, [2,nn,dim+1])
whh = tf.placeholder(tf.float32, [2,nn,nn+1])
why = tf.placeholder(tf.float32, [2,dim,nn+1])

feedforward = lambda p,w: tf.nn.sigmoid(tf.matmul(tf.concat([p,tf.ones([tf.shape(p)[0],1])],1), tf.transpose(w)))
def mlmc(wx,wh,wy):
    lh  = feedforward(x, wx)
    lh2 = feedforward(sh, wh)
    ly  = feedforward(sh2, wy)
    lyh = -log_prob(ly, tf.tile(y_,[ns**2,1]))
    lyx = tf.reduce_mean(tf.reshape(lyh, [ns,-1,dim]),0)
    
    ldy = tf.gradients(tf.reduce_mean(tf.reduce_sum(lyh,1)),wy)[0]
    ldh = tf.gradients(tf.stop_gradient(tf.reduce_sum(lyh,1,keep_dims=True))*log_prob(tf.tile(lh2,[ns,1]),sh2),wh)[0]
    ldx = tf.gradients(tf.stop_gradient(tf.reduce_sum(lyx,1,keep_dims=True))*log_prob(tf.tile(lh,[ns,1]),sh),wx)[0]
    return ldx, ldh, ldy

ldx, ldh, ldy = mlmc(wxh[1],whh[1],why[1])

lgd = [list(i) for i in dy] + [list(i) for i in dh] + [list(i) for i in dx]
lgd[2][0] -= ldx-wxh[0]
lgd[1][0] -= ldh-whh[0]
lgd[0][0] -= ldy-why[0]
for i in range(3):
    lgd[i][0] = tf.check_numerics(lgd[i][0], str(i))
lgdn = (tf.norm(lgd[2][0])+tf.norm(lgd[1][0]))/2.
ltrain = opt.apply_gradients(lgd)

In [ ]:
wxh2 = tf.placeholder(tf.float32, [2,nn,dim+1])
whh2 = tf.placeholder(tf.float32, [2,nn,nn+1])
why2 = tf.placeholder(tf.float32, [2,dim,nn+1])

ldx2, ldh2, ldy2 = mlmc(wxh2[1],whh2[1],why2[1])

lgd2 = [list(i) for i in dy] + [list(i) for i in dh] + [list(i) for i in dx]
lgd2[2][0] -= ldx2-wxh2[0] + ldx-wxh[0]
lgd2[1][0] -= ldh2-whh2[0] + ldh-whh[0]
lgd2[0][0] -= ldy2-why2[0] + ldy-why[0]
for i in range(3):
    lgd2[i][0] = tf.check_numerics(lgd2[i][0], str(i))
lgd2n = (tf.norm(lgd2[2][0])+tf.norm(lgd2[1][0]))/2.
l2train = opt.apply_gradients(lgd2)

### Train

In [ ]:
def fit_model(steps, filename, _lr, _ns, _mlmc):
    loss_rec = np.empty((1000, steps/1000))
    gd_rec = np.empty((1000, steps/1000))
    _tau = 2.
    _bsh, _bsx = 0., 0.
    
    _wxh = np.zeros([2,nn,dim+1])
    _whh = np.zeros([2,nn,nn+1])
    _why = np.zeros([2,dim,nn+1])
    _wxh2 = np.zeros([2,nn,dim+1])
    _whh2 = np.zeros([2,nn,nn+1])
    _why2 = np.zeros([2,dim,nn+1])    

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())            
        for i in range(steps):
            if i%100==0 and _mlmc:
                batch_   = data.next_batch(24, shuffle=True)[0]
                batch_xs = batch_[:, 0:dim]
                batch_ys = batch_[:, dim:2*dim]
                res = sess.run([nll, train, gd],
                                {x: batch_xs, y_: batch_ys, ns: _ns, lr: _lr, ts:i, tau:_tau,
                               })
                _wxh, _whh, _why = res[2][2], res[2][1], res[2][0]
                _wxh2, _whh2, _why2 = res[2][2], res[2][1], res[2][0]
                
            elif i%100==50 and _mlmc:
                batch_   = data.next_batch(12, shuffle=True)[0]
                batch_xs = batch_[:, 0:dim]
                batch_ys = batch_[:, dim:2*dim]
                res = sess.run([nll, ltrain, lgd],
                                {x: batch_xs, y_: batch_ys, ns: _ns, lr: _lr, ts:i, tau:_tau,
                                 wxh:_wxh, whh:_whh, why:_why,                                 
                               })
                _wxh2, _whh2, _why2 = res[2][2], res[2][1], res[2][0]
            
            else:
                batch_   = data.next_batch(1, shuffle=True)[0]
                batch_xs = batch_[:, 0:dim]
                batch_ys = batch_[:, dim:2*dim]
                res = sess.run([nll,
    #                            gdn, train],
    #                            mpgdn, mptrain],
    #                            bgdn, btrain],
    #                            gbtrain],
    #                            sttrain],
                                ltrain],
                                {x: batch_xs, y_: batch_ys, ns: _ns, lr: _lr, ts:i, tau:_tau,
                                 wxh:_wxh, whh:_whh, why:_why,
#                                 wxh2:_wxh2, whh2:_whh2, why2:_why2,
    #                             baselineh: _bsh, baselinex: _bsx,
                               })                
    #            _bsh,_bsx = res[-1], res[-2]
                
            loss_rec[i%1000, i/1000], gd_rec[i%1000, i/1000] = res[0], res[1]
            if (i+1)%1000==0:
                print (i+1)/1000, np.mean(loss_rec[:, i/1000], 0),

    np.save(filename, [loss_rec, gd_rec]) 
    return loss_rec, gd_rec, batch_xs, batch_ys, res[1]

In [ ]:
def fit_model(steps, filename, _lr, _ns, _mlmc):
    loss_rec = np.empty((1000, steps/1000))
    gd_rec = np.empty((2, 1000, steps/1000))
    
    _tau = 2.
    _bsh, _bsx = 0., 0.
    
    _wxh = np.zeros([2,nn,dim+1])
    _whh = np.zeros([2,nn,nn+1])
    _why = np.zeros([2,dim,nn+1])
    _wxh2 = np.zeros([2,nn,dim+1])
    _whh2 = np.zeros([2,nn,nn+1])
    _why2 = np.zeros([2,dim,nn+1])    

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())            
        for i in range(steps):

            batch_   = data.next_batch(24, shuffle=True)[0]
            batch_xs = batch_[:, 0:dim]
            batch_ys = batch_[:, dim:2*dim]
            res = sess.run([nll,
#                            gdn, train],
                            mpgdn, mptrain],
#                            spgdn, sptrain, ibtrain],
#                            bgdn, btrain],
#                            gbtrain],
#                            sttrain],
#                            ltrain],
                            {x: batch_xs, y_: batch_ys, ns: _ns, lr: _lr, ts:i, tau:_tau,
#                             wxh:_wxh, whh:_whh, why:_why,
#                                 wxh2:_wxh2, whh2:_whh2, why2:_why2,
#                             baselineh: _bsh, baselinex: _bsx,
                           })                
#            _bsh,_bsx = res[-1], res[-2]
                
            loss_rec[i%1000, i/1000] = res[0]
            gd_rec[:, i%1000, i/1000] = res[1]
            
            if (i+1)%1000==0:
                print (i+1)/1000, np.mean(loss_rec[:, i/1000], 0),

    np.save(filename, [loss_rec, gd_rec]) 
    return loss_rec, gd_rec, batch_xs, batch_ys, res[1]

In [ ]:
mp0 = fit_model(1000000, "mp_3e-4", 3e-4, 1, True)

In [ ]:
plt.style.use('default')
fig, ax = plt.subplots(1,2, figsize=(16,5))
ax[0].set_ylabel("Test negative log-likelihood")
ax[1].set_ylabel("Graidnet norm")

#ax[0].set_ylim(70,100)
#ax[0].set_ylim(20,40)
#ax[0].set_xlim(0,50)
ax[1].set_ylim(0,100)
for i in range(2):
#    ax[i].set_xlabel("Steps with the same # grads (1e4)")
#    ax[i].set_xlabel("Steps (1e4)")
    plot_banded_error(test[i], "mp", ax[i])
    
    
    
    fatlegend(ax[0])
    ax[i].grid(True)
plt.show()
#fig.savefig('nll2_2.png')

In [ ]:
batch_xs = ngd_svrg[2]
batch_ys = ngd_svrg[3]
prdt = ngd_svrg[4]
bs = 1

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,8))
up = np.concatenate([j for j in [np.reshape(i,[7,14]) for i in batch_xs]], 1)
dw = np.concatenate([j for j in [np.reshape(i,[7,14]) for i in batch_ys]], 1)
p1 = np.concatenate(np.split(np.concatenate([up,dw], 0), 1, 1), 0)
#dwy = np.concatenate([j for j in [np.reshape(i,[14,28]) for i in prdt]], 1)
#p2 = np.concatenate(np.split(np.concatenate([up,dwy], 0), bs, 1), 0)
ax[0].imshow(p1, cmap=plt.cm.gray, interpolation='none')
#ax[1].imshow(p2, cmap=plt.cm.gray, interpolation='none')
ax[0].grid(False)
#ax[1].grid(False)
plt.show()